[Reference](https://towardsdev.com/dependency-injection-in-python-c4248a096800)

#1. Via the constructor

In [2]:
class Client:
    def __init__(self, foo: Foo, bar: Bar):
        if foo is None:
            raise ValueError("foo must be provided")
        if bar is None:
            raise ValueError("bar must be provided")
        self.foo = foo
        self.bar = bar
        
    def do_something(self):
        self.foo.do_something()
        self.bar.do_something_else()

#2. Via setter method(s)

In [3]:
class Client:
    def __init__(self):
        self.foo = None
        self.bar = None

    def set_foo(self, foo: Foo):
        if foo is None:
            raise ValueError("foo must be provided")
        self.foo = foo

    def set_bar(self, bar: Bar):
        if bar is None:
            raise ValueError("foo must be provided")
        self.bar = bar

    def validate_dependencies(self):
        if self.foo is None:
            raise ValueError("foo has not been set")
        if self.bar is None:
            raise ValueError("bar has not been set")

    def do_something(self):
        self.validate_dependencies()

        self.foo.do_something()
        self.bar.do_something_else()

#3. Via the method to be invoked

In [4]:
class Client:
    def do_something(self, foo: Foo, bar: Bar):
        if foo is None:
            raise ValueError("foo must be provided")
        if bar is None:
            raise ValueError("bar must be provided")
        
        foo.do_something()
        bar.do_something_else()

In [5]:
from pymongo.mongo_client import MongoClient


class ApiClient:
    def __init__(self, host: str, port: int, database: str, collection: str):
        self.mongo_client = MongoClient(host, port)
        self.database = self.mongo_client.get_database(database)
        self.collection = self.database.get_collection(collection)

    def get_message(self, message_id: str) -> str:
        result = self.collection.find_one({"message_id": message_id})
        return result.get("message")

In [6]:
from unittest.mock import MagicMock, call

import pytest

from api_client import ApiClient


class TestApiClient:
    def test_get_message_should_return_correct_message(
        self, mock_mongo_client: MagicMock
    ):
        mock_collection = MagicMock()
        mock_collection.find_one.return_value = {
            "message_id": "some_id",
            "message": "some_message",
        }
        mock_database = MagicMock()
        mock_database.get_collection.return_value = mock_collection
        mock_mongo_client.return_value.get_database.return_value = mock_database

        api_client = ApiClient(
            host="some_hostname",
            port=4242,
            database="some_db",
            collection="some_collection",
        )

        message = api_client.get_message(message_id="some_id")

        assert message == "some_message"

    @pytest.fixture
    def mock_mongo_client(self, mocker) -> MagicMock:
        return mocker.patch("api_client.MongoClient")

In [7]:
from pymongo.collection import Collection


class ApiClient:
    def __init__(self, message_collection: Collection):
        self.message_collection = message_collection

    def get_message(self, message_id: str) -> str:
        result = self.message_collection.find_one({"message_id": message_id})
        return result.get("message")

In [9]:
from unittest.mock import MagicMock, call

from api_client import ApiClient


class TestApiClient:
    def test_get_message_should_return_correct_message(self):
        mock_collection = MagicMock()
        mock_collection.find_one.return_value = {
            "message_id": "some_id",
            "message": "some_message",
        }

        api_client = ApiClient(message_collection=mock_collection)
        message = api_client.get_message(message_id="some_id")

        assert message == "some_message"

    def test_get_message_should_call_find_one_on_mongo_client_correctly(self):
        mock_collection = MagicMock()

        api_client = ApiClient(message_collection=mock_collection)
        api_client.get_message(message_id="some_id")

        assert mock_collection.find_one.call_count == 1
        assert mock_collection.find_one.call_args == call({"message_id": "some_id"})

In [10]:
from pymongo.mongo_client import MongoClient

from api_client import ApiClient


def main():
    host = "some_host"          # these values are hardcoded
    port = 4242                 # with the assumption that
    database = "some_database"  # we can get these
    collection = "messages"     # from argparse or something

    mongo_client = MongoClient(host, port)
    database = mongo_client.get_database(database)
    message_collection = database.get_collection(collection)

    api_client = ApiClient(message_collection=message_collection)

    # api_client.get_message(...)


if __name__ == "__main__":
    main()